In [4]:
import numpy as np
import lmadatagen as ldg
print(ldg)

<module 'lmadatagen' from '/Users/SophusOlsen/Desktop/lmadatagen/lmadatagen/__init__.py'>


In [3]:
'''
    Fetch the raw video chunks stored as .json files 
    from the disc. We can use cached data if they files
    have been previously loaded.
'''

ret_from_disc = False
path_to_json = './json/partial/'

json_util = ldg.LMAJSONUtil(ldg.joint_keys, ldg.joint_indeces, ldg.meta_keys)

# Load the video chunks as a list.
segments = json_util.get_video_segments(
    path_to_json,
    not ret_from_disc,
    ret_from_disc
)

NameError: name 'ldg' is not defined

In [ ]:
'''
    Compute and extract motion features (based on LMA) from the motion
    data acquired through openpose. Various hyperparameters can be setup
    before extraction.
    
    We can setup hyperparameters for each available feature. The features are:
    [0] weight
    [1] time
    [2] space
    [3] flow
    [4] shape
    
    The hyperparameter is a LIST containig a 3D-tuple with the following meanings:
    
    TUPLE = (feature, effector, weights)
    LIST = [TUPLE0 ... TUPLE4]
    
    The LIST does not need to contain exactly 5 values, but it cannot exceed this limit.
    An effector is joint that is included in the computation of the feature. 
    The weights are a list of values (any numbers) corresponding to how important the
    correspoding effector is. 
    
'''

# Hyperparameter for feature computation
effectors_weights_ = [
    ('time', ['head', 'leftWrist', 'rightWrist'], [0.1, 0.45, 0.45]),
    ('weight', None, np.ones((len(ldg.joint_keys), ))),
]

gestures_per_batch_ = 7

# Setup the parameters (static function call)
ldg.LMARunner.set_lma_hyperparams(
    gestures_per_batch=gestures_per_batch_,
    effectors_weights=effectors_weights_
)

In [ ]:
# Extract the motion features by passing in the video
# segments we retrieved above
runner = ldg.LMARunner(segments)

In [ ]:
# Retrieve the features as a pandas "DataFrame"
motion_features = runner.dataframe()
print(motion_features)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

def process_motion_features(train, test):
    features = ['weight', 'time', 'space', 'flow', 'shape']
    
    cs = MinMaxScaler()
    trainX = cs.fit_transform(train[features])
    testX = cs.transform(test[features])
    return (trainX, testX)
    

(train, test) = train_test_split(motion_features, test_size=0.2, random_state=42)
trainX, testX = process_motion_features(train, test)

In [ ]:
def np_step(x, a):
    boolean_arr = np.zeros_like(x).astype(np.int32)
    for col, predicate in enumerate(a):
        boolean_arr[x[:, col] >= predicate, col] = 1
        
    return boolean_arr

def action_effort(effort_features, thresholds):
    binary_encoding = np_step(effort_features, zip(thresholds)) 
    decimal_encoding = binary_encoding.dot(
        1 << np.arange(binary_encoding.shape[-1] - 1, -1, -1)
    ).astype(np.int32)
    return decimal_encoding 

In [ ]:
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler

X = np.log10(np.array(motion_features[['weight', 'time', 'space']]))
X = StandardScaler().fit_transform(X)

t1 = np.median(X[:, 0])
t2 = np.median(X[:, 1])
t3 = np.median(X[:, 2])
ts = [t1, t2, t3]

targets = action_effort(X, ts)

pca = decomposition.PCA(n_components=2)
pca.fit(X)

X_2 = pca.transform(X)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

clusters = len(set(targets))

kmeans = KMeans(n_clusters=clusters)
kmeans.fit(X_2)

labels = kmeans.labels_
centers = kmeans.cluster_centers_

fig = plt.figure(0, figsize=(12, 6))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.scatter(X_2[:, 0], X_2[:, 1], c=targets)
ax2.scatter(X_2[:, 0], X_2[:, 1], c=labels)
ax2.scatter(centers[:, 0], centers[:, 1], c='k', s=200, alpha=0.5)

plt.show()

In [ ]:
from minisom import MiniSom

som = MiniSom(7, 7, 3, sigma=4, learning_rate=0.1)

print("Training...")
som.train_random(X, 20000, verbose=True)  # random training
print("\n...ready!")

In [ ]:
plt.figure(figsize=(8, 8))
for x, t in zip(X, targets):
    w = som.winner(x)
    plt.text(w[0]+.5,  w[1]+.5, str(t),
              color=plt.cm.rainbow(t / 10.), fontdict={'weight': 'bold',  'size': 11})
plt.axis([0, som.get_weights().shape[0], 0,  som.get_weights().shape[1]])
plt.show()